In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORTAÇÃO DA BASE DE SÓCIOS
socios_empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', dtype=str)
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,tempo_de_sociedade,cnpj_cpf_socio_mascarado
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,11,***923641**
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,11,***491386**
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,11,***718468**
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,9,***415907**
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,9,***022878**
...,...,...,...,...,...
1683063,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,10,***906288**
1683064,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,10,***950407**
1683065,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,10,***993174**
1683066,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,10,***288983**


In [3]:
# Remover coluna de tempo_de_sociedade, não necessária aqui
socios_empresas_rotuladas_df.drop(columns=['tempo_de_sociedade'], inplace=True)

### Junção com a base de pessoas que receberam recursos do governo federal:

In [4]:
# IMPORTAÇÃO DA BASE DE RECEBIMENTO DE RECURSOS DO GOVERNO
recursos_governo_df = pd.read_csv(r'C:\DATASETS\Recebimento Recursos Governo Federal\recebimento_recursos_governo-consolidado.csv', dtype=str)
recursos_governo_df

,Código Favorecido,Nome Favorecido,recebimento_recurso_governo_federal
0,***000000**,CARMELO CARLOMAGNO FILHO,-468.52
1,***000000**,ELISABETE SANTOS TASSINARI,4582.54
2,***000000**,LAUDETE IZABEL BORGIO DE ANDRADE,218793.06
3,***000000**,SABRINA DA COSTA OBIEDO,800.0
4,***000000**,THAIS SAUCO SOCCA,6400.0
...,...,...,...
3805117,98671597000362,MUNICIPIO DE VERANOPOLIS,208.1
3805118,98674401000121,SOCIEDADE ALFREDOCHAVENSE,4000.0
3805119,98675598000113,ASS DE PAIS E AMIGOS DOS EXCEPCIONAIS DE VERAN...,11160.0
3805120,98714876000102,ASSOCIACAO HOSPITAL NOSSA SENHORA DE POMPEIA,349600.0


In [5]:
print(recursos_governo_df['Código Favorecido'].str.len().value_counts())
pj_recursos_df = recursos_governo_df[recursos_governo_df['Código Favorecido'].str.len()==14]
pf_recursos_df = recursos_governo_df[recursos_governo_df['Código Favorecido'].str.len()==11]

11    3449497
14     355625
Name: Código Favorecido, dtype: int64


In [54]:
# Rotula os sócios que são PJ
socios_empresas_rotuladas_df['socio_recurso_recebido_governo_federal'] = socios_empresas_rotuladas_df.merge(pj_recursos_df, left_on='cnpj_cpf_socio', right_on='Código Favorecido', how='left')['recebimento_recurso_governo_federal'].astype(float)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_recurso_recebido_governo_federal.sum()

cnpj_cpf_socio
11    0.000000e+00
14    2.998973e+12
Name: socio_recurso_recebido_governo_federal, dtype: float64

Agora, vamos rotular as pessoas físicas:

Todos os CPFs já estão mascarados, como visto abaixo:

In [55]:
pf_recursos_df[(pf_recursos_df['Código Favorecido'].str[0:1] != '*')]

,Código Favorecido,Nome Favorecido,recebimento_recurso_governo_federal


In [56]:
# Rotula os sócios que são PF
socios_empresas_rotuladas_df['socio_recurso_recebido_governo_federal_pf'] = socios_empresas_rotuladas_df.merge(pf_recursos_df, left_on=['cnpj_cpf_socio_mascarado', 'nome_socio'], right_on=['Código Favorecido', 'Nome Favorecido'], how='left')['recebimento_recurso_governo_federal'].astype(float)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_recurso_recebido_governo_federal_pf.sum()

cnpj_cpf_socio
11    4.760264e+08
14    0.000000e+00
Name: socio_recurso_recebido_governo_federal_pf, dtype: float64

In [63]:
# Junta os marcadores:
socios_empresas_rotuladas_df.fillna({'socio_recurso_recebido_governo_federal': 0, 'socio_recurso_recebido_governo_federal_pf': 0}, inplace=True)
socios_empresas_rotuladas_df['socio_recurso_recebido_governo_federal'] = socios_empresas_rotuladas_df['socio_recurso_recebido_governo_federal']+socios_empresas_rotuladas_df['socio_recurso_recebido_governo_federal_pf']
socios_empresas_rotuladas_df.drop(columns=['socio_recurso_recebido_governo_federal_pf'], inplace=True)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_recurso_recebido_governo_federal.sum()

cnpj_cpf_socio
11    4.760264e+08
14    2.998973e+12
Name: socio_recurso_recebido_governo_federal, dtype: float64

In [70]:
socios_empresas_rotuladas_df[socios_empresas_rotuladas_df.socio_recurso_recebido_governo_federal > 0].groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len())['socio_recurso_recebido_governo_federal'].count()

cnpj_cpf_socio
11    19100
14     1638
Name: socio_recurso_recebido_governo_federal, dtype: int64

In [66]:
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,cnpj_cpf_socio_mascarado,socio_recurso_recebido_governo_federal
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,***923641**,0.00
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,***491386**,0.00
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,***718468**,0.00
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,***415907**,0.00
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,***022878**,0.00
...,...,...,...,...,...
1683063,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,***906288**,0.00
1683064,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,***950407**,0.00
1683065,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,***993174**,0.00
1683066,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,***288983**,132.96


In [71]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas-recebimento_recurso_governo_federal.csv', index = False)